In [1]:
import cv2
import numpy as np
import pickle
import os
import tensorflow as tf
from tensorflow import keras

print("Libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")

C:\Users\User\AppData\Roaming\Python\Python313\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


Libraries imported successfully!
TensorFlow version: 2.20.0


In [3]:
MODELS_DIR = "./Models"
MODEL_PATH = os.path.join(MODELS_DIR, "alexnet_face_recognition.h5")
LABELS_PATH = os.path.join(MODELS_DIR, "class_labels.pkl")
MEAN_IMG_PATH = os.path.join(MODELS_DIR, "mean_image.pkl")

IMG_SIZE = (227, 227)

if not os.path.exists(MODEL_PATH) or not os.path.exists(LABELS_PATH) or not os.path.exists(MEAN_IMG_PATH):
    print("❌ ERROR: Missing model files!")
    print("Please run 'Assignment2.ipynb' first to train and save the model.")
else:
    print("✅ All model files found.")

✅ All model files found.


In [4]:
print("Loading resources...")

model = keras.models.load_model(MODEL_PATH)
print("✅ Model loaded.")

with open(LABELS_PATH, 'rb') as f:
    class_labels_dict = pickle.load(f)
idx_to_label = {v: k for k, v in class_labels_dict.items()}
print(f"✅ Labels loaded: {list(class_labels_dict.keys())}")

with open(MEAN_IMG_PATH, 'rb') as f:
    mean_img = pickle.load(f)
print(f"✅ Mean image loaded (shape: {mean_img.shape})")

Loading resources...


✅ Model loaded.
✅ Labels loaded: ['Al-Amin', 'Imtiaz', 'Kaif', 'Shoaib']
✅ Mean image loaded (shape: (227, 227, 3))


In [6]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

if face_cascade.empty():
    print("❌ Error loading Haar Cascade XML!")
else:
    print("✅ Face detector initialized.")

✅ Face detector initialized.


In [7]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ Could not open webcam.")
else:
    print("✅ Webcam started. Press 'q' to quit.")
    print("Starting recognition loop...")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(60, 60))

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            face_roi = frame[y:y+h, x:x+w]
            
            try:
                face_resized = cv2.resize(face_roi, IMG_SIZE)
                
                face_rgb = cv2.cvtColor(face_resized, cv2.COLOR_BGR2RGB)
                
                face_float = face_rgb.astype(np.float32)
                
                face_norm = face_float - mean_img
                
                face_input = np.expand_dims(face_norm, axis=0)
                
                preds = model.predict(face_input, verbose=0)
                pred_idx = np.argmax(preds)
                confidence = preds[0][pred_idx]
                
                label_text = f"{idx_to_label[pred_idx]} ({confidence*100:.1f}%)"
                
                cv2.putText(frame, label_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                
            except Exception as e:
                print(f"Error processing face: {e}")

        cv2.imshow('Face Recognition (AlexNet)', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    print("✅ Webcam closed.")
    print("Thank you")

✅ Webcam started. Press 'q' to quit.
Starting recognition loop...
✅ Webcam closed.
Thank you
